In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! unzip /content/drive/MyDrive/'Colab Notebooks'/HandGestures/imgs.zip

In [12]:
import cv2
import numpy as np
import pandas as pd

import torch 
import torch.nn as nn
import torch.optim
from torchvision import datasets, transforms

In [ ]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

device = 'cpu'
if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    device = 'cuda'
    print('CUDA is available!  Training on GPU ...')

In [ ]:
torch.__version__

In [198]:
labels = {
    "ThumbUp": 0,
    "ThumbDown": 1,
    "Ok": 2,
    "Palm": 3
}

class HandGesturesDataset(torch.utils.data.Dataset):
  def __init__(self, csv_file, file_col, lbl_col, text2lbl):
    self.csv = pd.read_csv(csv_file)
    self.file_col = file_col
    self.lbl_col = lbl_col
    self.labels = text2lbl

  def __len__(self):
    return len(self.csv)

  def __getitem__(self, idx):
    img = cv2.imread(self.csv.loc[idx][self.file_col])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float).transpose(2, 0, 1)
    label = self.csv.loc[idx][self.lbl_col]

    return {"img": img, "label": self.labels[label]}

In [199]:
train = HandGesturesDataset("/content/drive/MyDrive/Colab Notebooks/HandGestures/imgs.csv",
                            "file", "label", labels)

In [200]:
trainloader = torch.utils.data.DataLoader(train, batch_size=1, shuffle=True)

In [201]:
from torchvision.models import resnet18, vgg16, inception_v3, densenet121, resnet50

def get_model(model_name, num_classes):
  if model_name == "resnet18":
    model = resnet18(num_classes=num_classes)
  elif model_name == "resnet50":
    model = resnet50(num_classes=num_classes)
  elif model_name == "inception_v3":
    model = inception_v3(num_classes=num_classes)
  return model


In [ ]:
model = get_model("resnet18", 4)
print(model)

In [203]:
def train_model(model, train_dl, optimizer, loss_fn, valid_dl = None, epochs=10):
  model.to(device)

  for ep in range(1, epochs + 1):
    train_loss, val_loss = [], []
    print("---" * 3, "Epoch", ep, "---" * 3)
    model.train()
    for batch in train_dl:
      optimizer.zero_grad()
      img = batch["img"].float().to(device)
      out = model(img)
      loss = loss_fn(out, batch["label"].to(device))
      train_loss.append(loss.item())
      loss.backward()
      optimizer.step()
    print("Training Loss: ", np.mean(train_loss))
    if valid_dl is not None:
      model = model.eval()
      for batch in valid_dl:
        img = batch["img"].to(device)
        out = model(img)
        loss = loss_fn(out, batch["label"].to(device))
        val_loss.append(loss.item())
      print("Validation Loss: ", np.mean(val_loss))

In [ ]:
model

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), 0.005)
loss_fn = torch.nn.CrossEntropyLoss()

train_model(model, trainloader, optimizer, loss_fn, epochs=3)

In [146]:
def test_model(model, testloader):
  model.eval()
  y_true, y_pred = [], []
  for img, lbl in testloader:
    img = img.to(device)
    out = model(img)
    y_true.extend(lbl)
    probabilities = torch.softmax(model(img), dim=1)
    predicted_class = torch.argmax(probabilities, dim=1)
    y_pred.extend(predicted_class.cpu())
  return y_true, y_pred

In [ ]:
y_true, y_pred = test_model(model, testloader)
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred, digits=3))